<a href='https://jupyter.org/'>Jupyter Notebook 2</a>

### ADVANCED PSE+ STAKEHOLDER SUMMIT I 2023: 
### Interactive Code Demonstration Using WaterTAP

#### Today's demonstration will show how to build, initialize, simulate, and optimize a flowsheet for multiperiod analysis.

## Overall approach to multiperiod flowsheets
<p align="center">
  <img src="assets/MP_Framework.png" width="80%">
</p>

## Part 1: Build, setup, and simulate the multiperiod RO+PV+Battery flowsheet

<p align="center">
  <img src="assets/RO_PV_Batt.svg" width="80%">
</p>

## Multiperiod Setup

### Quick and high-level overview of setting up steady-state and surrogate parts
### Import Pyomo, IDAES, and WaterTAP packages

In [1]:
# Pyomo imports
from pyomo.environ import ConcreteModel, Objective, Var, value, units as pyunits

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.apps.grid_integration.multiperiod.multiperiod import MultiPeriodModel
from idaes.core.solvers.get_solver import get_solver


### Build Model

In [2]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

### Import and define the Reverse Osmosis unit model

In [3]:
# from steady_state_flowsheets.simple_RO_unit import ROUnit
# m.fs.RO = ROUnit()

In [4]:
from watertap_contrib.reflo.code_demos.steady_state_flowsheets.ro_system import *
from watertap.property_models.NaCl_prop_pack import NaClParameterBlock
from watertap.costing import WaterTAPCosting

m.fs.properties = NaClParameterBlock()
m.fs.RO = FlowsheetBlock(dynamic=False)
build_system(m)
# build_ro(m, m.fs.RO, number_of_stages=1)
# add_connections(m)
# add_constraints(m)
# set_ro_scaling_and_params(m)
# set_operating_conditions(m)


            =======> BUILDING RO SYSTEM <=======            

    (type=<class 'pyomo.core.base.expression.ScalarExpression'>) on block
    fs.RO with a new Component (type=<class
    'pyomo.core.base.expression.ScalarExpression'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().

       =======> SETTING OPERATING CONDITIONS <=======       

2023-12-19 16:50:33 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.RO.feed.properties[0.0].flow_mass_phase_comp[Liq,H2O]
2023-12-19 16:50:33 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.RO.feed.properties[0.0].flow_mass_phase_comp[Liq,NaCl]
2023-12-19 16:50:33 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.RO.product.properties[0.0].flow_mass_phase_comp[Liq,H2O]
2023-12-19 16:50:33 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.RO.product.properties[0.0].flow_mass_phase_comp[Liq,NaCl]
2023-1

### This simple RO unit will assume a steady-state system with constant production and power demand
        ROUnit.product = 6000 # m3/day
        ROUnit.power_demand = 1 # MW

### Import and define the Battery model

In [5]:
from watertap_contrib.reflo.code_demos.steady_state_flowsheets.battery import BatteryStorage 
# From DISPATCHES 
m.fs.battery = BatteryStorage()

<p align="center">
  <img src="assets/var_table.png" width="70%">
</p>

### Load PV surrogate

In [6]:
from idaes.core.surrogate.pysmo_surrogate import PysmoSurrogate
PV_surrogate = PysmoSurrogate.load_from_file('assets/demo_surrogate.json')

2023-12-19 16:50:33 [INFO] idaes.core.surrogate.pysmo_surrogate: Decode surrogate. type=rbf
Default parameter estimation method is used.

Parameter estimation method:  algebraic
Gaussian basis function is used.
Basis function:  gaussian
Regularization done:  True


<p align="center">
  <img src="assets/solar_cycle2.png" width="50%">
</p>

##### The PV surrogate will predict the energy production of a given PV system based on:
* PV system design size [kW] (peak power )
* Day of the year
* Hour of the Day

### Define critical variables and add energy balance constraints in one time step/ steady-state conditions

In [7]:
from watertap_contrib.reflo.code_demos.steady_state_flowsheets.system import *
define_system_vars(m)
add_steady_state_constraints(m)

### System-level variables include:
| Var                       | Description                                    | Unit |
| :----:                    | :----:                                    |:----:|
| pv_to_ro           | Energy supplied to the RO coming from PV | kW  |
| grid_to_ro          | Energy supplied to the RO coming from grid | kW |
| curtailment   | PV curtailment                 | kW |
| elec_price                   | Electricity Price                                 | $/kWh  |
| pv_gen                   | Energy in                                 | kWh  |

### The steady state constraints:

<p align="center">
  <img src="assets/Balance_dark.png" width="80%">
  <img src="assets/constraint_dark.png" width="80%">
</p>

### Let's fix a few variables for this initial solve

In [8]:
m.fs.battery.nameplate_energy.fix(8000) # Battery capacity [kWh]
m.fs.battery.nameplate_power.fix(400) # Battery power [kW]
m.fs.pv_gen.fix(700) # PV generation [kW]

In [9]:
solver = get_solver()
init_system(m)
m.fs.battery.initialize()
results = solver.solve(m)



-------------------- INITIALIZING SYSTEM --------------------


2023-12-19 16:50:33 [INFO] idaes.init.fs.feed: Initialization Complete.
2023-12-19 16:50:33 [INFO] idaes.init.fs.primary_pump.control_volume: Initialization Complete
2023-12-19 16:50:33 [INFO] idaes.init.fs.primary_pump: Initialization Complete: optimal - Optimal Solution Found


-------------------- INITIALIZING RO SYSTEM --------------------


2023-12-19 16:50:33 [INFO] idaes.init.fs.RO.feed: Initialization Step Complete.
2023-12-19 16:50:33 [INFO] idaes.init.fs.RO.stage[1].feed: Initialization Step Complete.
2023-12-19 16:50:33 [INFO] idaes.init.fs.RO.stage[1].module.feed_side: Initialization Complete
2023-12-19 16:50:34 [INFO] idaes.init.fs.RO.stage[1].module: Initialization Complete: optimal - Optimal Solution Found
2023-12-19 16:50:34 [INFO] idaes.init.fs.RO.stage[1].permeate: Initialization Step Complete.
2023-12-19 16:50:34 [INFO] idaes.init.fs.RO.stage[1].retentate: Initialization Step Complete.
2023-12-19 16:50

In [10]:
print_system_results(m)

fs.water_recovery                        0.5        dimensionless
fs.feed_salinity                         35.0       dimensionless
fs.feed_flow_mass                        150.0      kg/s
fs.perm_flow_mass                        1.0        kg/s
fs.cross_flow_velocity                   0.2        m/s
fs.pv_to_ro                              229.6      kW
fs.grid_to_ro                            755.3      kW
fs.curtailment                           328.9      kW
fs.elec_generation                       1,000.0    kW
fs.pv_gen                                700.0      kW
fs.pv_size                               1,000.0    kW
fs.ro_elec_req                           1,000.0    kW
fs.electricity_price                     0.1        USD_2021


## Part 2: Build and Solve Multiperiod Flowsheet
### Refresh - approach to multiperiod flowsheets
<p align="center">
  <img src="assets/MP_Framework_2.png" width="80%">
</p>

### Link variables across time

In [11]:
def get_pv_ro_variable_pairs(t1, t2):
    """
    This function returns pairs of variables that need to be connected across two time periods

    Args:
        t1: current time block
        t2: next time block

    Returns:
        None
    """
    return [
        (t1.fs.battery.state_of_charge[0], t2.fs.battery.initial_state_of_charge),
        (t1.fs.battery.energy_throughput[0], t2.fs.battery.initial_energy_throughput),
        (t1.fs.battery.nameplate_power, t2.fs.battery.nameplate_power),
        (t1.fs.battery.nameplate_energy, t2.fs.battery.nameplate_energy),
        ]

<p align="center">
  <img src="assets/linking.png" width="70%">
</p>

### Define multiperiod flowsheet and create each time instance

<p align="center">
  <img src="assets/Multiperiod.png" width="100%">
</p>

### Provide some costing constraints

<p align="center">
<img src="assets/costing_constraints.png" width="50%">
</p>

In [12]:
def create_multiperiod_pv_battery_model(
        n_time_points= 24,
        surrogate = None):
    
    # create the multiperiod object
    '''The `MultiPeriodModel` class helps transfer existing steady-state
    process models to multiperiod versions that contain dynamic time coupling'''
    mp = MultiPeriodModel(
        n_time_points=n_time_points,
        process_model_func= steady_state_flowsheet,
        linking_variable_func= get_pv_ro_variable_pairs,
        initialization_func= initialize_system,
        unfix_dof_func= unfix_dof,
    )

    # process_model_func - This is our steady-state or single time instance flowsheet
    # unfix_dof - This is where we fix or unfix battery size, power, etc...
    # initialize_system - This is where we initialize the battery
    
    # Flowsheet options is where we define the input options for the steady-state flowsheet
    flowsheet_options={ t: { 
                            "pv_gen": eval_surrogate(surrogate, design_size = 1000, Day = t//24, Hour = t%24),
                            "electricity_price": get_elec_tier(Hour = t%24),} 
                            for t in range(n_time_points)
    }

    # Build a multi-period capable model using user-provided functions
    mp.build_multi_period_model(
        model_data_kwargs=flowsheet_options)
    
    # Fix the initial state of charge to zero
    mp.blocks[0].process.fs.battery.initial_state_of_charge.fix(0)

    # Set capital costs and the objective function that apply across all time periods
    add_costing_constraints(mp)

    # Total Costs = Battery Costs + PV Costs + RO Costs
    # LCOW = Total Costs / RO Production [$/m3]

    return mp

## Simulation

In [13]:
mp = create_multiperiod_pv_battery_model(surrogate=PV_surrogate)

[+   0.00] Beginning the formulation of the multiperiod optimization problem.
2023-12-19 16:50:35 [INFO] idaes.apps.grid_integration.multiperiod.multiperiod: ...Constructing the flowsheet model for blocks[0]

            =======> BUILDING RO SYSTEM <=======            

    (type=<class 'pyomo.core.base.expression.ScalarExpression'>) on block
    fs.RO with a new Component (type=<class
    'pyomo.core.base.expression.ScalarExpression'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().

       =======> SETTING OPERATING CONDITIONS <=======       

2023-12-19 16:50:35 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.RO.feed.properties[0.0].flow_mass_phase_comp[Liq,H2O]
2023-12-19 16:50:35 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.RO.feed.properties[0.0].flow_mass_phase_comp[Liq,NaCl]
2023-12-19 16:50:35 [WARNING] idaes.core.util.scaling: Missing scaling factor for f

KeyboardInterrupt: 

In [ ]:
results = solver.solve(mp)
print_results(mp)

In [ ]:
print([value(mp.blocks[i].process.fs.battery.state_of_charge[0]) for i in range(24)])

In [ ]:
from watertap_contrib.reflo.code_demos.util.visualize import *
create_long_plot(mp)

## Now let's optimize the battery

#### Unfix 
* Battery Nameplate Power
* Battery Nameplate Energy

In [ ]:
mp_optimized = optimize_multiperiod_pv_battery_model(surrogate=PV_surrogate)
results = solver.solve(mp_optimized)
print_results(mp_optimized)

In [ ]:
create_plot(mp_optimized)

### We can also run this simulation over longer timescales

In [ ]:
mp_week = create_multiperiod_pv_battery_model(n_time_points=24*7, surrogate=PV_surrogate)
results = solver.solve(mp_week)
print_results(mp_week)

In [ ]:
create_long_plot(mp_week)